In [0]:
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType,StructField,DateType,TimestampType,StringType,IntegerType,DecimalType
from pyspark.sql.functions import year, month, dayofmonth,to_timestamp,to_date,split,substring,col,when
from pyspark.sql.functions import udf

In [0]:
storage_acct = "xxxxxxxxxx"
container_name = "xxxxxx"
blob_key = "xxxxxxxxxxxxxxxxxxxx"

In [0]:
mountName = "walmartSalesETL"
mounts = [str(i) for i in dbutils.fs.ls('/mnt/')] 

if "FileInfo(path='dbfs:/mnt/" + mountName + "/', name='" + mountName + "/', size=0)" in mounts:
  print("mount already created")
else:
  dbutils.fs.mount(
    source = "wasbs://" + container_name + "@" + storage_acct +".blob.core.windows.net",
    mount_point = "/mnt/walmartSalesETL",
    extra_configs = {"fs.azure.account.key." + storage_acct + ".blob.core.windows.net":blob_key})
  

mount already created

In [0]:
%run "./WalmartSales-modules-dev"

In [0]:
def main():
  try:
    inputFilePath = "/mnt/walmartSalesETL/inputFile/"
    salesDf = spark.read.option("header","true").csv(inputFilePath)
    
    salesDf.show(5)
    
    # Handle Null Customer values
    salesDf = replaceNullCustomerID(salesDf)
  
    # Handle Null Description
    salesDf = replaceNullDescription(salesDf)
  
    # Add Quarter column
    salesDf = addcolumnQuarter(salesDf)
  
    # Add Invoice Type column
    salesDf = addcolumnInvoiceType(salesDf)
  
    # Split files by UK and Others
    salesDf_uk,salesDf_others = filterDf(salesDf)
  
    # Write UK files to output folder
    outPath = "/mnt/walmartSalesETL/outputFile/sales_uk"
    salesDf_uk.write.mode("append").csv(outPath)

    # Write others files to output folder
    outPath = "/mnt/walmartSalesETL/outputFile/sales_others"
    salesDf_others.write.mode("append").csv(outPath)
  
  except Exception as e:
    print("Exception occured " + str(e))

In [0]:
try:
  main()
except Exception as e:
  print("Exception occured " + str(e))

+-------+---------+--------------------+--------+--------------+-----+-----------+--------------+
Invoice|StockCode| Description|Quantity| InvoiceDate|Price|Customer ID| Country|
+-------+---------+--------------------+--------+--------------+-----+-----------+--------------+
 489434| 85048|15CM CHRISTMAS GL...| 12|12/1/2009 7:45| 6.95| 13085|United Kingdom|
 489434| 79323P| PINK CHERRY LIGHTS| 12|12/1/2009 7:45| 6.75| 13085|United Kingdom|
 489434| 79323W| WHITE CHERRY LIGHTS| 12|12/1/2009 7:45| 6.75| 13085|United Kingdom|
 489434| 22041|"RECORD FRAME 7""...| 48|12/1/2009 7:45| 2.1| 13085|United Kingdom|
 489434| 21232|STRAWBERRY CERAMI...| 24|12/1/2009 7:45| 1.25| 13085|United Kingdom|
+-------+---------+--------------------+--------+--------------+-----+-----------+--------------+
only showing top 5 rows